In [116]:
import string
import re
import pandas as pd 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk import pos_tag
from pymorphy3 import MorphAnalyzer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
import torch




## Preprocess texts

In [61]:
df = pd.read_csv('../wb_parse_subject=406after_images.csv')

In [62]:
df['описание'][0]

'Чай матча зелёный Япония премиальный . Полезные свойства : оказывает мощное тонизирующее действие , повышает активность мозга , работоспособность , уравновешивает эмоциональный фон и успокаивает нервную систему . Напиток 100% органические заряжает энергией ( повышенное содержание кофеина в паре с L-теанином ) . Японские продукты нормализуют обмен веществ . Японская еда снижает риск появления и развития сердечно-сосудистых заболеваний . Чай порошковый натуральный стимулирует мозговую деятельность , заметно улучшает концентрацию внимания , благодаря чему увеличиваются объемы усваиваемой информации в короткие сроки .\nМатча органическая как заваривать : Приготовьте на 70 мл горячей воды ( температура 80-85 С ) одну ложку порошка залейте водой , взбивая смесь венчиком . Готовность определяется по образованию устойчивой пены и полной однородности консистенции . Чтобы добиться нормальной температуры , после закипания чайника подождите 6-7 мин . Напитки мача от Vegannova Веганова выбирают ст

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3505 entries, 0 to 3504
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          3505 non-null   int64  
 1   id                  3505 non-null   int64  
 2   бренд               3491 non-null   object 
 3   id бренда           3505 non-null   int64  
 4   название            3505 non-null   object 
 5   цена                3505 non-null   float64
 6   бренд.1             3491 non-null   object 
 7   рейтинг             3505 non-null   float64
 8   количество оценок   3505 non-null   int64  
 9   айди поставщика     3505 non-null   int64  
 10  рейтинг поставщика  3505 non-null   float64
 11  в наличии           3505 non-null   int64  
 12  ссылка на фото      3505 non-null   object 
 13  описание            3501 non-null   object 
 14  категория рейтинга  3505 non-null   object 
dtypes: float64(3), int64(6), object(6)
memory usage: 410.9+

In [64]:
df['описание'] = df['описание'].fillna('Описания нет')
df['бренд'] = df['бренд'].fillna('Бренда нет')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3505 entries, 0 to 3504
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          3505 non-null   int64  
 1   id                  3505 non-null   int64  
 2   бренд               3505 non-null   object 
 3   id бренда           3505 non-null   int64  
 4   название            3505 non-null   object 
 5   цена                3505 non-null   float64
 6   бренд.1             3491 non-null   object 
 7   рейтинг             3505 non-null   float64
 8   количество оценок   3505 non-null   int64  
 9   айди поставщика     3505 non-null   int64  
 10  рейтинг поставщика  3505 non-null   float64
 11  в наличии           3505 non-null   int64  
 12  ссылка на фото      3505 non-null   object 
 13  описание            3505 non-null   object 
 14  категория рейтинга  3505 non-null   object 
dtypes: float64(3), int64(6), object(6)
memory usage: 410.9+

In [65]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    return tokens

In [66]:
def preprocess_text(row):

    text = str(row).lower()
    
    # deleting punctuation
    punc = str.maketrans('', '', string.punctuation)
    text_no_punct = text.translate(punc)

    # lemmatizing 
    lematized_text = lemmatize(text_no_punct)
    
    return lematized_text

In [67]:
preprocessed_df = df.copy()
preprocessed_df['описание'] = df['описание'].apply(preprocess_text)

In [68]:
preprocessed_df

,Unnamed: 0,id,бренд,id бренда,название,цена,бренд.1,рейтинг,количество оценок,айди поставщика,рейтинг поставщика,в наличии,ссылка на фото,описание,категория рейтинга
0,0,17870700,VeganNova,82980,Матча чай японский 100г,277.0,VeganNova,4.7,18856,65087,4.8,4,https://basket-02.wbbasket.ru/vol178/part17870...,"[чай, матч, зелёный, япония, премиальный, поле...","(4.52, 4.76]"
1,1,17136591,Herbarus,40309,Чай травяной Спокойствие Баланс 24пак,190.0,Herbarus,4.9,4977,90373,4.5,29,https://basket-02.wbbasket.ru/vol171/part17136...,"[яркий, неповторимый, вкус, натуральный, ингре...","(4.76, 5.0]"
2,2,66555836,Дивеевское подворье,956062,Чага березовая натуральная дробленая,219.0,Дивеевское подворье,4.9,1008,561218,4.9,18,https://basket-04.wbbasket.ru/vol665/part66555...,"[уважаемый, покупатель, ложка, комплект, чага,...","(4.76, 5.0]"
3,3,163596778,E&B world of tea,525840,Чай черный фруктовый в подарок,183.0,E&B world of tea,4.8,11708,352249,4.8,9,https://basket-11.wbbasket.ru/vol1635/part1635...,"[чай, чёрный, листовой, фруктовоцветочный, вку...","(4.76, 5.0]"
4,4,17136594,Herbarus,40309,Чай травяной Ассорти пакет. 24 шт,193.0,Herbarus,4.9,4977,90373,4.5,31,https://basket-02.wbbasket.ru/vol171/part17136...,"[яркий, неповторимый, вкус, натуральный, ингре...","(4.76, 5.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3500,4894,11877743,Mitchel Tea,66926,Митчел Чай Ройбуш (Roibos) Апельсиновый крем,239.0,Mitchel Tea,4.9,335,49566,4.8,8,https://basket-01.wbbasket.ru/vol118/part11877...,"[африканский, ройбуш, очень, интересный, расте...","(4.76, 5.0]"
3501,4895,13809820,Tea grow,88633,"Чай Зеленый Китайский Сенча, 100г",225.0,Tea grow,4.8,1129,60357,4.7,6,https://basket-01.wbbasket.ru/vol138/part13809...,"[самый, натуральный, популярный, сорт, классич...","(4.76, 5.0]"
3502,4897,143376286,TeaTale,649125,Чай подарочный листовой набор,197.0,TeaTale,4.9,2502,181516,4.8,38,https://basket-10.wbbasket.ru/vol1433/part1433...,"[подарочный, вкусный, крупнолистовый, чай, цве...","(4.76, 5.0]"
3503,4898,22925297,SurprizBox,124512,"Подарочный набор чая""Антистресс"" чай и мед",1157.0,SurprizBox,4.8,458,106348,4.8,32,https://basket-02.wbbasket.ru/vol229/part22925...,"[подарочный, наборв, состав, сорт, премиальный...","(4.76, 5.0]"


## Training embeddings

Придумать метрику оценивания док2века 

In [70]:
# creating tagged data
tagged_data = [TaggedDocument(words=d, tags=[str(i)]) for i, d in enumerate(preprocessed_df['описание'])]

# creating model
model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=10)

# receiving vecs 
vectors = [model.infer_vector(doc.words) for doc in tagged_data]


In [79]:
df['описание'][100]
test = 'Цейлонский черный чай, состоит из ломанных листьев с большим содержанием типсов (нераспустившихся почек). Типсы придают чаю более нежный вкус и тонкий аромат Почка и первый лист идут в подарок.'
test = preprocess_text(test)
test

['цейлонский',
 'чёрный',
 'чай',
 'состоять',
 'ломать',
 'лист',
 'больший',
 'содержание',
 'типёс',
 'нераспустившийся',
 'почка',
 'типёс',
 'придавать',
 'чай',
 'нежный',
 'вкус',
 'тонкий',
 'аромат',
 'почка',
 'первый',
 'лист',
 'идти',
 'подарок']

In [100]:
infer = model.infer_vector(test)
similar_docs = model.docvecs.most_similar([infer], topn=10)
print("Самые близкие документы:")
for doc_id, similarity in similar_docs:
    print(f"Document ID: {doc_id}, Similarity: {similarity}")


Самые близкие документы:
Document ID: 2002, Similarity: 0.8404029607772827
Document ID: 1457, Similarity: 0.8381947875022888
Document ID: 1476, Similarity: 0.8375450968742371
Document ID: 799, Similarity: 0.8363845348358154
Document ID: 1075, Similarity: 0.8335375189781189
Document ID: 2514, Similarity: 0.8313694596290588
Document ID: 1917, Similarity: 0.8309014439582825
Document ID: 1065, Similarity: 0.8286433219909668
Document ID: 1359, Similarity: 0.8277016282081604
Document ID: 735, Similarity: 0.8275507092475891


/var/folders/pd/fjb9sj5x6t3dnxtpzxxzkv6c0000gn/T/ipykernel_19037/2407805844.py:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_docs = model.docvecs.most_similar([infer], topn=10)


In [115]:
from numpy import dot
from numpy.linalg import norm

# Предположим, что model - ваша обученная модель Doc2Vec
# Предположим, что doc_id = 100

# Получить вектор для документа 100 из модели
doc100_vector = model.docvecs[0]

# Предсказанный вектор для вашего текста
# Предположим, что inferred_vector - предсказанный вектор для вашего текста

# Рассчитать косинусное расстояние между предсказанным вектором и вектором для документа 100
similarity = dot(infer, doc100_vector) / (norm(infer) * norm(doc100_vector))

print(f"Уровень схожести между предсказанным вектором и вектором документа 100: {similarity}")

Уровень схожести между предсказанным вектором и вектором документа 100: 0.5409044027328491


/var/folders/pd/fjb9sj5x6t3dnxtpzxxzkv6c0000gn/T/ipykernel_19037/3121007888.py:8: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  doc100_vector = model.docvecs[0]


Test bert for embeddings

In [ ]:
П